In [18]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
 
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate
 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

## nonregularized_model

In [19]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()
 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_12 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)       

In [23]:
batch_size = 64
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
# train the network
model.fit(x_train, y_train, epochs=40, batch_size=batch_size)

# validate the model on test dataset to determine generalization
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))


Epoch 1/40
782/782 [==============================] - 16s 20ms/step - loss: 0.5771 - accuracy: 0.7973
Epoch 2/40
782/782 [==============================] - 15s 20ms/step - loss: 0.5677 - accuracy: 0.8003
Epoch 3/40
782/782 [==============================] - 15s 20ms/step - loss: 0.5494 - accuracy: 0.8057
Epoch 4/40
782/782 [==============================] - 16s 20ms/step - loss: 0.5482 - accuracy: 0.8104
Epoch 5/40
782/782 [==============================] - 15s 20ms/step - loss: 0.5397 - accuracy: 0.8116
Epoch 6/40
782/782 [==============================] - 15s 20ms/step - loss: 0.5243 - accuracy: 0.8158
Epoch 7/40
782/782 [==============================] - 15s 20ms/step - loss: 0.5156 - accuracy: 0.8198
Epoch 8/40
782/782 [==============================] - 16s 20ms/step - loss: 0.5051 - accuracy: 0.8235
Epoch 9/40
782/782 [==============================] - 15s 20ms/step - loss: 0.4984 - accuracy: 0.8255
Epoch 10/40
782/782 [==============================] - 15s 20ms/step - loss: 0.487

## regularized_model

In [21]:
weight_decay = 1e-4
l2regularized_model = Sequential()
l2regularized_model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
l2regularized_model.add(Activation('elu'))
l2regularized_model.add(BatchNormalization())
l2regularized_model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
l2regularized_model.add(Activation('elu'))
l2regularized_model.add(BatchNormalization())
l2regularized_model.add(MaxPooling2D(pool_size=(2,2)))
l2regularized_model.add(Dropout(0.2))
 
l2regularized_model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
l2regularized_model.add(Activation('elu'))
l2regularized_model.add(BatchNormalization())
l2regularized_model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
l2regularized_model.add(Activation('elu'))
l2regularized_model.add(BatchNormalization())
l2regularized_model.add(MaxPooling2D(pool_size=(2,2)))
l2regularized_model.add(Dropout(0.3))
 
l2regularized_model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
l2regularized_model.add(Activation('elu'))
l2regularized_model.add(BatchNormalization())
l2regularized_model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
l2regularized_model.add(Activation('elu'))
l2regularized_model.add(BatchNormalization())
l2regularized_model.add(MaxPooling2D(pool_size=(2,2)))
l2regularized_model.add(Dropout(0.4))
 
l2regularized_model.add(Flatten())
l2regularized_model.add(Dense(num_classes, activation='softmax'))
 
l2regularized_model.summary()
 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_18 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_19 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 32)       

In [ ]:
batch_size = 64
l2regularized_model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
# train the network
l2regularized_model.fit(x_train, y_train, epochs=40, batch_size=batch_size)

# validate the model on test dataset to determine generalization
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

Epoch 1/40
782/782 [==============================] - 16s 20ms/step - loss: 0.2479 - accuracy: 0.9256
Epoch 2/40
782/782 [==============================] - 16s 20ms/step - loss: 0.2463 - accuracy: 0.9267
Epoch 3/40
782/782 [==============================] - 16s 20ms/step - loss: 0.2404 - accuracy: 0.9284
Epoch 4/40
782/782 [==============================] - 16s 20ms/step - loss: 0.2435 - accuracy: 0.9273
Epoch 5/40
782/782 [==============================] - 16s 20ms/step - loss: 0.2355 - accuracy: 0.9294
Epoch 6/40
782/782 [==============================] - 16s 20ms/step - loss: 0.2336 - accuracy: 0.9320
Epoch 7/40
782/782 [==============================] - 16s 20ms/step - loss: 0.2349 - accuracy: 0.9304
Epoch 8/40
782/782 [==============================] - 16s 20ms/step - loss: 0.2303 - accuracy: 0.9326
Epoch 9/40
782/782 [==============================] - 16s 20ms/step - loss: 0.2325 - accuracy: 0.9321
Epoch 10/40
782/782 [==============================] - 16s 20ms/step - loss: 0.225